In [ ]:
!pip install gradio

In [ ]:
!pip install gradio transformers librosa soundfile

import gradio as gr
import librosa
import numpy as np
import textwrap
import torch
from transformers import pipeline

# Load models
asr_pipeline = pipeline("automatic-speech-recognition", model="openai/whisper-small")
summarizer = pipeline("summarization", model="sshleifer/distilbart-cnn-12-6",
                      device=0 if torch.cuda.is_available() else -1)
qna_pipeline = pipeline("question-answering", model="distilbert-base-uncased-distilled-squad",
                        device=0 if torch.cuda.is_available() else -1)

# --- NEW: Chunking for long audio ---
def chunk_audio(audio_array, sr, chunk_len_sec=30):
    chunk_size = sr * chunk_len_sec
    for start in range(0, len(audio_array), chunk_size):
        end = min(start + chunk_size, len(audio_array))
        yield audio_array[start:end]

# ✅ Transcription function now handles long audio
def transcribe_audio(audio):
    if audio is None:
        return "No audio input."
    try:
        file_path = audio if isinstance(audio, str) else audio.name
        audio_data, sr = librosa.load(file_path, sr=16000)

        transcripts = []
        for chunk in chunk_audio(audio_data, sr):
            result = asr_pipeline({"array": chunk, "sampling_rate": sr})
            transcripts.append(result["text"])

        return " ".join(transcripts)

    except Exception as e:
        return f"Error in transcription: {str(e)}"

# Paragraph summarizer
def abstractive_summarize(text, chunk_size=700):
    if not text.strip():
        return "No text to summarize."
    chunks = textwrap.wrap(text, chunk_size)
    partial_summaries = []
    for chunk in chunks:
        summary = summarizer(chunk, max_length=180, min_length=60, do_sample=False)[0]['summary_text']
        partial_summaries.append(summary)
    combined = " ".join(partial_summaries)
    final_summary = summarizer(combined, max_length=250, min_length=100, do_sample=False)[0]['summary_text']
    return final_summary

# Bullet-point summary
def bullet_summary(text):
    if len(text.split()) < 40:
        return "Text too short for a detailed summary."
    prompt = "Summarize the following transcript into 5 clear bullet points:\n" + text
    result = summarizer(prompt, max_length=180, min_length=60, do_sample=False)[0]['summary_text']
    return "\n".join(["• " + line.strip() for line in result.split(".") if line.strip()])

# Summarize from transcript
def summarize_transcript(transcript):
    if not transcript.strip():
        return "No transcript to summarize.", "No transcript to summarize."
    para = abstractive_summarize(transcript)
    bullets = bullet_summary(transcript)
    return para, bullets

# Q&A from summary
def answer_question(question, summary_context):
    if not summary_context.strip():
        return "No summary available."
    if not question.strip():
        return "Please enter a question."
    answer = qna_pipeline(question=question, context=summary_context)
    return answer["answer"]

# Gradio UI
with gr.Blocks() as demo:
    #gr.HTML("<style>body { overflow: hidden; }</style>")
    gr.Markdown("## Vocal ")

    with gr.Column():
            audio_input = gr.Audio(sources=["microphone", "upload"], type="filepath", label="🎙️ Upload/Record Audio")
    with gr.Row():
        with gr.Column():
                transcribed_text = gr.Textbox(label="📝 Transcribed Text", lines=10)
                transcribe_btn = gr.Button("🎤 Transcribe Audio")
        with gr.Column():
                summarized_text = gr.Textbox(label="📄 Paragraph Summary", lines=10)
                summarize_btn = gr.Button("📝 Summarize Transcript")
        with gr.Column():
                bullet_text = gr.Textbox(label="📌 Bullet-Point Summary", lines=10)
                bullet_btn = gr.Button("📝 Bulletpoints")
        transcribe_btn.click(transcribe_audio, inputs=audio_input, outputs=transcribed_text, show_progress=True)
        summarize_btn.click(summarize_transcript, inputs=transcribed_text, outputs=summarized_text, show_progress=True)
        bullet_btn.click(summarize_transcript, inputs=transcribed_text, outputs=bullet_text, show_progress=True)
    with gr.Column():
        question_input = gr.Textbox(label="❓ Ask a question about the summary")
        answer_output = gr.Textbox(label="✅ Answer")
        ask_button = gr.Button("Get Answer")
        ask_button.click(fn=answer_question, inputs=[question_input, summarized_text], outputs=answer_output, show_progress=True)
demo.launch()

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Device set to use cpu
Device set to use cpu
Device set to use cpu


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://ff2b4e6e60c6a46d04.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
